In [3]:
import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

from sklearn.preprocessing import quantile_transform

%matplotlib inline

In [5]:
full = pd.read_csv('data/gen_clean.csv')

In [13]:
mis = pd.read_csv('data/gen_mi_best_neurons.csv', index_col=0);
mis['neuron'] = mis.index

In [14]:
from dionysus import Filtration, homology_persistence, init_diagrams, Simplex
from collections import Counter
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def get_cycles_for_filtration(filtration, end_time=np.inf):
    m = homology_persistence(filtration, prime=2)
    dgms = init_diagrams(m, filtration)
    cycles = []
    for i, dgm in enumerate(dgms):
        for pt in dgm:
            cycles.append((i, pt.birth, pt.death))
            #print((i, pt.birth,pt.death))
    cycles = pd.DataFrame(cycles, columns=['dimension', 'birth_time', 'death_time'])
    cycles['living_time'] = cycles['death_time'] - cycles['birth_time']
    inf_life = (cycles['living_time'] == np.inf)
    cycles['living_time'][inf_life] = end_time - cycles['birth_time'][inf_life]
    return cycles.sort_values('living_time', ascending=False)



def homologies(df, number_of_ap):
    # time - simplexes that live > time%
    # number_of_ap = (time/100) * len(df)
    #number_of_ap=90
    #print(number_of_ap)

    df = quantile_transform(df)
    time = 10
    filtration = Filtration()
    for alpha in np.arange(0.50, 0.99, 0.01):
        #print("alpha", alpha)
        neuron_activity = data > alpha
        
        complexes = []
        sims = np.sum(neuron_activity * (2**np.arange(neuron_activity.shape[1])), axis=1)
        #print(Counter(sims))
        for sim, cnt in Counter(sims).items():
            if cnt > number_of_ap:
                complexes.append(np.where(np.array(list(bin(int(sim))[2:][::-1])) == '1')[0])
        
        all_simplexes = []
        for compl in complexes:
            for s in list(powerset(compl))[1:]:
                all_simplexes.append(s)
                
                
        all_simplexes = sorted(list(set(all_simplexes)), key=lambda x: len(x))
        for sim in all_simplexes:
            filtration.append(Simplex(sim, alpha))
        #print(filtration)
        #print()
    cycles = get_cycles_for_filtration(filtration, 0.95)
    #cycles.columns = [f"{col}_{number_of_ap/df.shape[0]}" for col in cycles.columns]
    
    return cycles.reset_index(drop=True)

In [16]:
for n in [2, 3, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 500][:]:
    print(n)
    mi_best_neurons = mis[:10]['neuron']
    data = full.drop(['x', 'y'], axis=1)[mi_best_neurons.index]
    df = homologies(data, n)
    print(df.T)
    print()

2
                0
dimension    0.00
birth_time   0.50
death_time    inf
living_time  0.45

3
                0
dimension    0.00
birth_time   0.50
death_time    inf
living_time  0.45

5
                0
dimension    0.00
birth_time   0.50
death_time    inf
living_time  0.45

10
                0     1     2     3     4
dimension    0.00  6.00  6.00  6.00  6.00
birth_time   0.50  0.50  0.50  0.50  0.50
death_time    inf   inf   inf   inf   inf
living_time  0.45  0.45  0.45  0.45  0.45

20
               0     1     2     3     4     5     6     7     8     9   ...  \
dimension    0.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00  2.00  ...   
birth_time   0.50  0.59  0.60  0.61  0.61  0.61  0.61  0.61  0.62  0.62  ...   
death_time    inf   inf   inf   inf   inf   inf   inf   inf   inf   inf  ...   
living_time  0.45  0.36  0.35  0.34  0.34  0.34  0.34  0.34  0.33  0.33  ...   

               35    36    37    38    39    40    41    42    43    44  
dimension    2.00  2.00  1.00